# Lab | Customer Analysis Final

## Problem (case study)

Input of this analyses is data from marketing.

In [ ]:
# Data Description
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model           # skearn is for machine learning 
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import math                       
import warnings                  
warnings.filterwarnings('ignore')
#in_file = 'marketing_customer_analysis.csv'        # official data for the Lab
in_file = 'marketing_customer_analysis_round2.csv'  # with data to be standardized
customers = pd.DataFrame(pd.read_csv(in_file))
print('Input: marketing_customer_analysis.csv')
print('Number of rows:', customers.shape[0])
print('Number of columns', customers.shape[1])
print(' ')
print(customers.dtypes)
pd.set_option('display.max_columns', None)  # all columns in jupyter
display(customers.head())

### Goal
The goal of the analysis is to create a model to predict if behaviour of customers with certain characteristics. The objective of the underlying data is to understand customer demographics and buying behavior. In the data we can find several information about the customers, such as: where they live, if they responded to earlier campaigns, personal information and customer value information.

## Getting Data

In [ ]:
# Read the .csv file
# done in previous chapter

## Cleaning/Wrangling/EDA 

<details><summary>▶ Theory: 10 ways to drop columns in dataframes</summary>
<p>

### 10 Different ways to drop columns:
**Delete with del**<br />
<span style="color:#444; font-family: 'Courier'; background-color: #F7F7F7">
del df['Locations']</span>
    
**Drop the label 'Locations'**<br />
<span style="color:#444; font-family: 'Courier'; background-color: #F7F7F7">
df.drop(labels='Locations', axis=1)</span>
    
**With columns parameter**<br /> 
The value of the axis parameter need not be passed.<br />
<span style="color:#444; font-family: 'Courier'; background-color: #F7F7F7">
df.drop(columns='Founder')</span>
    
**Multiple columns**<br />
<span style="color:#444; font-family: 'Courier'; background-color: #F7F7F7">
df.drop(labels=['Locations', 'Founder'], axis=1)</span>
    
**Pass a list of column names to the columns parameter**<br />
<span style="color:#444; font-family: 'Courier'; background-color: #F7F7F7">
df.drop(columns=['Founder', 'Locations'])</span>
    
**drop multiple columns, using an index**<br />
<span style="color:#444; font-family: 'Courier'; background-color: #F7F7F7">
df.drop(df.columns[[1, 3]], axis=1)</span>

**drop multiple columns, using index as variable**<br />
<span style="color:#444; font-family: 'Courier'; background-color: #F7F7F7">
drop_i = Index(['Name', 'Locations', 'States'], dtype='object')< br/>
df.drop(df.columns[drop_i], axis=1)</span>
    
**Pass column names to the loc indexing method**<br />
<span style="color:#444; font-family: 'Courier'; background-color: #F7F7F7">
df.drop(df.loc[:, ['Locations', 'Founder']], axis=1)</span>

**Drop columns with certain patterns using .loc method**<br />
You can also pass name patterns as label names to the loc index. Using name patterns, you can remove all the columns from a DataFrame which have the specified pattern in them. Function .startswith() is a string function which is used to check if a string starts with the specified character or not.<br />
<span style="color:#444; font-family: 'Courier'; background-color: #F7F7F7">
df.drop(df.loc[:, df.columns[df.columns.str.startswith('F')]], axis=1)</span>
    
**Drop tables, EXCEPT the ones**<br />
And pass the column names which are to be retained<br />
<span style="color:#444; font-family: 'Courier'; background-color: #F7F7F7">
df.drop(df.columns.difference(['Name', 'States', 'Founding Year']), axis=1)</span>
    
**The pop method**<br />
Is used to remove the specified column from the DataFrame and return the removed column as a pandas Series. Pass the name of the column which is to be removed and return it as a pandas Series.<br />
<span style="color:#444; font-family: 'Courier'; background-color: #F7F7F7">
founder = df.pop('Founder')<br />
print(founder)<br />
print('\n')  # Escape character to print an empty new line<br />
print(df)<br /></span>

</p>
</details>

In [ ]:
# Change headers names
customers.columns = customers.columns.map(lambda x : x.replace("-", "_").replace(" ", "_"))
# still not satisfied with EmploymentStatus
customers = customers.rename(columns={"EmploymentStatus":"Employment_Status"})
# Drop the first column
customers = customers.drop(labels='Unnamed:_0', axis=1)
customers.columns

### Deal with NaN values

In [ ]:
# Deal with NaN values (1)
# Apparently there are no records in the input file that are without value. 
# To illustrate and to make it more fun, I have added the input file from 
# round 2 to the final Lab. This one is called 
# marketing_customer_analysis_round2.csv
print(customers.isna().sum())
print('Number of rows:', customers.shape[0])
print('Number of columns', customers.shape[1])

In [ ]:
# Dealing with NaN values (2)

# Column State: fill NaN value of a certain row with a given value
customers['State'] = customers['State'].fillna('USA')

In [ ]:
# Dealing with NaN values (3)

# Colum Response: fill boolean column with No (sounds like least possible consequence)
# display(customers[customers['Response'].isna()==True].head(60))    # test line to see some values
# customers.loc[:,['Response']].head(60)                             # alternative test line
customers['Response'] = customers['Response'].fillna('No')

In [ ]:
# Dealing with NaN values (4)

# Column Months_Since_Last_Claim: fill the NaN fields of this column with mean value of the rest
mean_Months_Since_Last_Claim = round(customers['Months_Since_Last_Claim'].mean(),1)
customers['Months_Since_Last_Claim'] = customers['Months_Since_Last_Claim'].fillna(mean_Months_Since_Last_Claim)

In [ ]:
# Dealing with NaN values (5)

# Column Number_of_Open_Complaints: with mean value rounded whole numbers
mean_Number_of_Open_Complaints = round(customers['Number_of_Open_Complaints'].mean(),0)
customers['Number_of_Open_Complaints'] = customers['Number_of_Open_Complaints'].fillna(mean_Number_of_Open_Complaints)

In [ ]:
# Dealing with NaN values (6)

# Columns Vehicle_Class and Vehicle_Size 
# it seems like both columns have NaN together so I treat them equally
# customers.loc[86:100,['Vehicle_Class','Vehicle_Size']].head(60)
# want to know most common Vehicle_Size
# customers['Vehicle_Size'].unique()
# aparently 3 options L / M / S
# customers['Vehicle_Size'].value_counts()
# decide to fill Vehicle_Size with Medsize
customers['Vehicle_Size'] = customers['Vehicle_Size'].fillna('Medsize')
# customers['Vehicle_Class'].value_counts()
# decide to fill Vehicle_Size with Four-Door Car
customers['Vehicle_Class'] = customers['Vehicle_Class'].fillna('Four-Door Car')

In [ ]:
# Dealing with NaN values (7)

# Column Vehicle_Class | Vehicle_Size | Vehicle_Type
# pd.set_option('display.max_columns', None)    # see all columns, since Months_Since_Last_Claim was hidden
# customers[customers['Vehicle_Size'].isna()==True].head(60)    # test line to see some values
# seems that all NaN on Class and Size are vehicle type A
# customers.loc[customers['Vehicle_Type'] == 'A']
# customers.loc[customers['Vehicle_Size'] == 'A']
# customers.loc[:,['Vehicle_Class','Vehicle_Size','Vehicle_Type']].head(60)
# Looking at Vehicle_Type seems that this field is really bad on data quality,
# since it's either A or NaN. It doesn't really give more or better info than Vehicle_Class 
# and Vehicle_Size together, therefore:
# Decide to eliminate column Vehicle_Type
exist_vt = "Vehicle_Type" in customers                    # avoid errors at re-run code
if exist_vt==True:
    customers = customers.drop(['Vehicle_Type'], axis=1)  # drop a single column 

In [ ]:
# Dealing with NaN values (Final check)
round(customers.isna().sum()/len(customers),4)*100

In [ ]:
# Categorical Features
cat_cols = customers.select_dtypes('object').columns
print('Categorical Features in this dataframe are:')
print(' ')
for col in cat_cols:
    print(col)

In [ ]:
# Numerical Features
print('Numerical Features in this dataframe are:')
print(' ')
num_cols = customers._get_numeric_data()
for col in num_cols:
    print(col)

### Exploratory Data Analysis

Exploratory data analysis is an approach of analyzing data sets to summarize their main characteristics, often using statistical graphics and other data visualization methods: for seeing what the data can tell us beyond the formal modeling or hypothesis testing task.

In [ ]:
# EDA: Use some of these to explore:
#
# customers.info(memory_usage='deep')
# print(customers.shape)
# display(customers.head(60))
# display(customers)
# Exploratory Data Analysis (1)
# 
# Explore 1: Effective_To_Date - convert this column to a real date? 
# Explore 2: Income - what do we do with odd incomes (negative or 0)?
#
# Convert Effective_To_Date to date format
customers['Effective_To_Date'].dtype
customers['Effective_To_Date'] = pd.to_datetime(customers['Effective_To_Date'], errors='coerce')
customers['Effective_To_Date'].dtype
# 
# Put rows in seperate df where Income is 0 
customers_Inc0 = customers.loc[customers.Income == 0]   # perhaps for later use as df
# customers_Inc0  # 2787 rows × 24 columns
customers = customers.loc[customers.Income > 0]
# customers  # 8123 rows × 24 columns

In [ ]:
# EDA: visualization on Response Rate
%matplotlib inline

sns.countplot('Response', data=customers)
plt.ylabel('Total number of Response')
plt.show()

In [ ]:
# EDA: visualization on Response by the sales channel

plt.figure(figsize=(8,4))
sns.countplot('Response', hue='Sales_Channel', data=customers)
plt.ylabel('Response by Sales Channel')
plt.show()

In [ ]:
# EDA: visualization on Response Rate by the Total Claim Amount

plt.figure(figsize=(12,6))
sns.boxplot(y='Total_Claim_Amount' , x='Response', data=customers)
plt.ylabel('Response by Total Claim Amount')
plt.show()

In [ ]:
# EDA: visualization on Response Rate by Income

plt.figure(figsize=(12,6))
sns.boxplot(y='Income' , x='Response', data=customers)
plt.ylabel('Response by Income')
plt.show()

In [ ]:
# First rough exploration with the "plot bomb"
sns.pairplot(customers)

In [ ]:
# Visualize all numeric columns with a loop for displot
# %%time
df1 = customers.select_dtypes([int, float])
for i, col in enumerate(df1.columns):
    plt.figure(i)
    sns.displot(x=col, data=df1)

In [ ]:
# Visualize all numeric columns with a loop for histplot
df1 = customers.select_dtypes([int, float])
for i, col in enumerate(df1.columns):
    plt.figure(i)
    sns.histplot(x=col, data=df1, color="orange")

In [ ]:
# Visual exploration of correlation with the heatmap
correlations_matrix = customers.corr()
sns.heatmap(correlations_matrix, annot=True)
plt.show()

In [ ]:
# Conclusions based on the correlation matrix:
# Some correlations that stand out:
    # Monthly_Premium_Auto <--> Customer_Lifetime_Value (0.4)
    # Monthly_Premium_Auto <--> Total_Claim_Amount (0.63)
    # total_claim_amount <--> Income (negative correlation -0.36)
# The rest seems to be neutral
# Regarding total_claim_amount it's interesting to study possibility
# to predict with the use of Monthly_Premium_Auto and Income
# There are no features to be dropped due to very high correlation
# Since highest corr is 0.63

## Processing Data 

### Dealing with outliers

In [ ]:
# Dealing with outliers
shape_before = customers.shape    # checksum 
print(shape_before)
# Check what happens with notmality when outliers are removed, for:
# - Monthly_Premium_Auto 
# - Customer_Lifetime_Value
# - Total_Claim_Amount

In [ ]:
# # Dealing with outliers - Monthly_Premium_Auto

In [ ]:
# sns.boxplot(x=customers['Monthly_Premium_Auto'])
# plt.show()
# # sns.displot(customers['Monthly_Premium_Auto'])
# # plt.show()

In [ ]:
# iqr = np.percentile(customers['Monthly_Premium_Auto'],75) - np.percentile(customers['Monthly_Premium_Auto'],25)
# upper_limit = np.percentile(customers['Monthly_Premium_Auto'],75) + 1.5*iqr
# lower_limit = np.percentile(customers['Monthly_Premium_Auto'],25) - 1.5*iqr
# print('Inter Quartile Range:', iqr, 'Lower Limit (25)', lower_limit, 'Upper Limit (75):', upper_limit)
# # Next line is to actually remove the outliers for Monthly_Premium_Auto
# customers = customers[(customers['Monthly_Premium_Auto']>lower_limit) & (customers['Monthly_Premium_Auto']<upper_limit)]
# print('Customers without outliers in Monthly_Premium_Auto:', customers.shape, 'Outliers Deleted:', shape_before[0]-customers.shape[0] )
# sns.displot(customers['Monthly_Premium_Auto'])
# plt.show()

In [ ]:
# On closer inspection, however, these data do not provide a good normal distribution. 
# Decision: comment out the steps for dealing with outliers for Monthly_Premium_Auto
# And re-run the cells

In [ ]:
# Dealing with outliers - Customer_Lifetime_Value

In [ ]:
# sns.boxplot(x=customers['Customer_Lifetime_Value'])
# plt.show()
# sns.displot(customers['Customer_Lifetime_Value'])
# plt.show()

In [ ]:
# iqr = np.percentile(customers['Customer_Lifetime_Value'],75) - np.percentile(customers['Customer_Lifetime_Value'],25)
# upper_limit = np.percentile(customers['Customer_Lifetime_Value'],75) + 1.5*iqr
# lower_limit = np.percentile(customers['Customer_Lifetime_Value'],25) - 1.5*iqr
# print('Inter Quartile Range:', iqr, 'Lower Limit (25)', lower_limit, 'Upper Limit (75):', upper_limit)
# # # Next line is to actually remove the outliers for Monthly_Premium_Auto
# customers = customers[(customers['Customer_Lifetime_Value']>lower_limit) & (customers['Customer_Lifetime_Value']<upper_limit)]
# print('Customers without outliers in Monthly_Premium_Auto:', customers.shape, 'Outliers Deleted:', shape_before[0]-customers.shape[0] )
# sns.displot(customers['Customer_Lifetime_Value'])
# plt.show()

In [ ]:
# On closer inspection, however, these data do not provide a good normal distribution. 
# Decision: comment out the steps for dealing with outliers for Customer_Lifetime_Value
# And re-run the cells

In [ ]:
# Dealing with outliers - Total_Claim_Amount

In [ ]:
sns.boxplot(x=customers['Total_Claim_Amount'])
plt.show()
sns.displot(customers['Total_Claim_Amount'])
plt.show()

In [ ]:
iqr = np.percentile(customers['Total_Claim_Amount'],75) - np.percentile(customers['Total_Claim_Amount'],25)
upper_limit = np.percentile(customers['Total_Claim_Amount'],75) + 1.5*iqr
lower_limit = np.percentile(customers['Total_Claim_Amount'],25) - 1.5*iqr
print('Inter Quartile Range:', iqr, 'Lower Limit (25)', lower_limit, 'Upper Limit (75):', upper_limit)
# # Next line is to actually remove the outliers for Monthly_Premium_Auto
customers = customers[(customers['Total_Claim_Amount']>lower_limit) & (customers['Total_Claim_Amount']<upper_limit)]
print('Customers without outliers in Monthly_Premium_Auto:', customers.shape, 'Outliers Deleted:', shape_before[0]-customers.shape[0] )
sns.displot(customers['Total_Claim_Amount'])
plt.show()

In [ ]:
# Unlike the outlier elimination operations for Monthly_Premium_Auto and Customer_Lifetime_Value, 
# this operation seem to have a good effect on Total_Claim_Amount
customers.shape

### Normalization
Normalization is the process of scaling individual samples to have unit norm. This process can be useful if you plan to use a quadratic form to quantify the similarity of any pair of samples.

In [ ]:
# Normalization of Total_Claim_Amount (1)
sns.scatterplot(x=customers['Total_Claim_Amount'], y=customers['Monthly_Premium_Auto'])
plt.show()

In [ ]:
# Normalization of Total_Claim_Amount (2)
# First remove target from the columns
y = customers['Monthly_Premium_Auto']
X = customers.drop(['Monthly_Premium_Auto'], axis=1)
X_num = X.select_dtypes(include = np.number)
X_cat = X.select_dtypes(include = object)

In [ ]:
# Normalization of Total_Claim_Amount (3a)
# Normalizing data: make data range from 0 - 1, instead of from min to max
MinMax_transf = MinMaxScaler().fit(X_num)
x_normalized = MinMax_transf.transform(X_num)
print(x_normalized.shape)
pd.DataFrame(x_normalized, columns=X_num.columns)

In [ ]:
# Normalization of Total_Claim_Amount (3b)
# scaling standard scaler: make data normal distributed with mean=0 and std=1
standard_transf = StandardScaler().fit(X_num)
x_standardized = standard_transf.transform(X_num)
print(x_standardized.shape)
pd.DataFrame(x_standardized, columns=X_num.columns)

### Encoding
<br>

<details><summary>▶ Theory Encoding</summary>
<p>

To convert categorical features to integer codes in order to use them in models, we can use different encoders. To consider:
* One Hot Encoder
* Label encoding

</p>
</details>

In this analysis we use One Hot Encoder, following these steps:
* Step 1: separate the features from the labels (into y)
* Step 2: separate the numeric columns (cust_X_num) from the categorical (cust_X_cat)
* Step 3: Hot label Encoding serveral categorical columns via a for loop
* Step 4: normalizing num data
* Step 5: merging cust_X_num and cust_X_cat

In [ ]:
# Encoding Categorical Data - step 1
# Separate the features from the labels
cust_y = customers['Total_Claim_Amount']
cust_X = customers.drop(['Total_Claim_Amount'], axis=1)
# cust_X.head()
# cust_y.head()

In [ ]:
# Encoding Categorical Data - step 2
# Categorical features and numerical ones are going to be treated differently
cust_X_num = X.select_dtypes(include = np.number)

# In the R2 Model Validation appears that target Total_Claim_Amount should be eliminated, otherwise
# there will be an R2 of 1.0 which is maximum score and not correct
cust_X_num = cust_X_num.drop(labels="Total_Claim_Amount", axis=1)

cust_X_cat = X.select_dtypes(include = object)
cust_X_cat = cust_X_cat.drop(columns=['Customer'])     # doesn't do anything useful and causes problems
# cust_X_num.head()
# cust_X_cat.head()

In [ ]:
# Encoding Categorical Data - step 3
# Hot label Encoding
onehot_pd = pd.DataFrame()
for column in cust_X_cat:
    # print("Processing column:", column)
    cust_encoder = OneHotEncoder().fit(cust_X_cat[[column]])
    cust_encoded = cust_encoder.transform(cust_X_cat[[column]]).toarray()
    onehot_encoded = pd.DataFrame(cust_encoded, columns=cust_encoder.categories_)
    onehot_pd = pd.concat([onehot_pd, onehot_encoded], axis=1)
onehot_pd.head()

In [ ]:
# Encoding Categorical Data - step 4
# Normalizing num data (again)
rene = MinMaxScaler().fit(cust_X_num)
cust_X_num_n = rene.transform(cust_X_num)
print(cust_X_num_n.shape)
cust_X_num_n = pd.DataFrame(cust_X_num_n, columns=cust_X_num.columns)
display(cust_X_num_n)

In [ ]:
# Encoding Categorical Data - step 5
# merging X_num and X_cat
cust_X_n = pd.concat([cust_X_num_n, onehot_pd], axis=1)
cust_X_n.shape
#display(cust_X_n.head(60))

### Train set and test set splitting
<br>

<details><summary>▶ Theory Split-testing</summary>
<p>

Splitting into train-set and test-set: In order to not give our model the oportunity to cheat, it must accurately guess the values in the "fresh" dataset that it never saw before: X_train, X_test, y_train, y_test. Training is for building, testing is for validating. Naming conventions are # y_train en Y_set is meant as target. When Train is 80%, test is 20%. Random state gives THE SAME random set (for illustration in course), sometimes for testing. This could be any number this option is to fix the randomizer.

</p>
</details>

In [ ]:
# Splitting into train set and test set.
X_train, X_test, y_train, y_test = train_test_split(cust_X_n, cust_y, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
X_train.head()

In [ ]:
y_train.head()

## Modeling

In [ ]:
# Apply model.
lm = linear_model.LinearRegression()
lm.fit(X_train,y_train)

## Model Validation

In [ ]:
# R2
predictions = lm.predict(X_train)
print("R2 score:", r2_score(y_train, predictions))

In [ ]:
# MSE
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_train, predictions)
print('MSE'mse)

In [ ]:
# OK, SOMETHING IS WRONG HERE WITH THE MSE ... I CALL IT A DAY, BECAUSE I CAN'T SEE WHAT I DID WRONG
# LACK OF EXPERIENCE WITH STATISTICS TO SOLVE THIS ON MY OWN - NEED SUPPORT!!

In [ ]:
# RMSE



In [ ]:
# MAE


## Reporting

In [ ]:
# Reporting



<details><summary>▶ Theory</summary>
<p>

Dit is een test

</p>
</details>

In [ ]:
print(customers.shape)
display(customers.head(60))

In [ ]:
display(customers)